In [1]:
# 최신순으로 최근 3달 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
from datetime import datetime

In [2]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [3]:
def dateCal(str):
    date = str.split(" ")
    year = int(date[0][:-1])
    month = int(date[1][:-1])
    day = int(date[2][:-1])
    
    return datetime(year, month, day)

In [8]:
cafeNames = [
    '빵굽는커피공장',
    '글루미',
    '마마다이닝'
]

cafeNums = [
    '053-422-3833', # 빵굽는커피공장
    '0507-1331-5449', # 글루미
    '0507-1374-2778' # 마마다이닝
]

cafeIDs = [
    '19675007', # 빵굽는커피공장
    '1431782782', # 글루미
    '1696700298' # 마마다이닝
]

In [ ]:
# for cafeName, cafeNum in zip(cafeNames, cafeNums):
#     cafeID = findCafe(cafeNum, cafeName)
#     cafeIDs.append(cafeID)
#     print(cafeID)

In [9]:
def clickMore(driver):
    now = datetime.now()

    while True: 
            try: 
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                time.sleep(1)
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                time.sleep(1) 
                
                date = driver.find_element(By.CSS_SELECTOR, '.eCPGL > li:nth-last-child(1) > div:nth-last-child(1) > span > span:nth-last-child(1)').text
                
                dt = dateCal(date)
                diff = (now-dt).total_seconds()//86400
                
                if diff > 90: # 맨 마지막 리뷰 일자가 90일 이전이면 멈춤
                    break
                
                driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click()
                time.sleep(2)           
                
                
            except NoSuchElementException: 
                print('-더보기 버튼 모두 클릭 완료-') 
                break 

In [14]:
def scrapReviews(html, cafename, cafelist, datelist, reviewlist):
    dom = BeautifulSoup(html, 'lxml')

    reviews = dom.select('.YeINN > .ZZ4OK > a > span')
    dates = dom.select('.YeINN > div:nth-last-child(1) > span > span:nth-last-child(1)')



    try: 
        for date, review in zip(dates, reviews): 
            
            day = dateCal(date.text)
            text = review.text
        
            cafelist.append(cafename)
            datelist.append(day)
            reviewlist.append(text)

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        print("네이버 리뷰 없음" )

In [11]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options=options)

In [ ]:
cafes = []
days = []
re = []

for cafeID, cafeName in zip(cafeIDs, cafeNames):
    finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'
    driver.get(finalurl)

    clickMore(driver)

    html = driver.page_source

    scrapReviews(html, cafeName, cafes, days, re)
    print(f'{cafeName} 리뷰 수집 완료')

driver.quit()

In [28]:
driver.quit()

In [21]:
driver = webdriver.Chrome()

In [22]:
finalurl = f'https://m.place.naver.com/restaurant/{cafeIDs[2]}/review/visitor?reviewSort=recent'
driver.get(finalurl)

clickMore(driver)

html = driver.page_source

scrapReviews(html, cafeNames[2], cafes, days, re)
print(f'{cafeName} 리뷰 수집 완료')

마마다이닝 리뷰 수집 완료


In [23]:
df = pd.DataFrame({'cafeName':cafes,
                   'date': days,
                   'review': re
                   })

In [24]:
df.head()

,cafeName,date,review
0,빵굽는커피공장,2022-12-02,조용하고 분위기 넘 좋아요
1,빵굽는커피공장,2022-11-19,조아용
2,빵굽는커피공장,2022-11-08,ㅇ
3,빵굽는커피공장,2022-11-02,굿
4,빵굽는커피공장,2022-10-29,ㅇ


In [25]:
df.tail()

,cafeName,date,review
317,마마다이닝,2022-09-09,내용 더보기
318,마마다이닝,2022-09-09,이쁘네요.
319,마마다이닝,2022-09-09,♡♡
320,마마다이닝,2022-09-09,빙수랑 티랑 잘 마시고 갑니다!
321,마마다이닝,2022-09-09,늦은시간까지 시내에서 하는 카페 별로 없는데 여긴 늦은시간에도 하고 디저트도 다양하...


In [30]:
df.to_csv('./data/test_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
data = pd.read_csv('./data/test_data.csv', encoding='utf-8-sig')